In [2]:
### This script looks at all files
### 1) Checks deleted files for md5 related fields and clears them
###                         for qc metric and changes status of qc metric object to deleted
###                         clears the qc metric field
###                         for workflows and deletes all of them
### 2) Checks other files for workflows and deleted old workflows
###                                     and deleted problematic ones (status or rev)
### takes around 20 min

from core.utils import Tibanna
from core import ff_utils
from datetime import datetime

# set enviroment and key/connection
env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

def get_wfr_report(wfrs):
    wfr_report = []
    for wfr_data in wfrs:
        wfr_rep = {}
        """For a given workflow_run_sbg item, grabs details, uuid, run_status, wfr name, date, and run time"""
        wfr_uuid = wfr_data['uuid']
        wfr_data = ff_utils.get_metadata(wfr_uuid, connection = ff)
        wfr_status = wfr_data['run_status']
        try:
            wfr_name = wfr_data['title'].split(' run ')[0]
        except:
            print('ProblematicCase')
            print(wfr_data['uuid'], wfr_data.get('display_title', 'no title'))
            continue
        wfr_time = datetime.strptime(wfr_data['date_created'],'%Y-%m-%dT%H:%M:%S.%f+00:00')
        run_hours = (datetime.utcnow()-wfr_time).total_seconds()/3600
        wfr_name_list = wfr_data['title'].split(' run ')[0].split('/')
        wfr_name = wfr_name_list[0]
        try:
            wfr_rev = wfr_name_list[1] 
        except:
            wfr_rev = "0"

        output_files = wfr_data.get('output_files',None)
        output_uuids = []
        if output_files:
            for i in output_files:
                if i.get('value', None):
                    output_uuids.append(i['value'])

        wfr_rep = {'wfr_uuid': wfr_data['uuid'],
                   'wfr_status': wfr_data['run_status'],
                   'wfr_name': wfr_name,
                   'wfr_rev': wfr_rev,
                   'wfr_date': wfr_time,
                   'run_time': run_hours,
                   'status': wfr_data['status'],
                   'outputs': output_uuids}
        wfr_report.append(wfr_rep)
    wfr_report = sorted(wfr_report, key=lambda k: (k['wfr_date'], k['wfr_name']))
    return wfr_report

    
def printTable(myDict, colList=None):
    """ Pretty print a list of dictionaries Author: Thierry Husson"""
    if not colList: colList = list(myDict[0].keys() if myDict else [])
    myList = [colList] # 1st row = header
    for item in myDict: myList.append([str(item[col] or '') for col in colList])
    colSize = [max(map(len,col)) for col in zip(*myList)]
    formatStr = ' | '.join(["{{:<{}}}".format(i) for i in colSize])
    myList.insert(1, ['-' * i for i in colSize]) # Seperating line
    for item in myList: print(formatStr.format(*item))
        


In [5]:
print 'started at', datetime.utcnow()
delete_workflows = raw_input("Do you want to delete old workflowruns (if not, only report will be displayed (y/n))")

# what kind of files should be searched for worflow run inputs, use url compatible naming

# accepted workflows
# workflow name, accepted revision numbers (0 if none), accetable run time (hours)
workflow_details = [
                    ['md5', ['0'], 24],
                    ['fastqc-0-11-4-1', ['0', '1'], 72],
                    ['hi-c-processing-parta-juicer', ['25','26'], 150],
                    ['hi-c-processing-partb', ['31', '34', '38'],150],
                    ['hi-c-processing-partc', ['3', '8'], 150],
                    ['add-hic-normvector-to-mcool', ['3'], 150],
                    ['extract-mcool-normvector-for-juicebox', ['100'], 150],
                    ['extract-mcool-normvector-for-juicebox-1', ['1'], 150],
                    ['bwa-mem', ['0'], 150],
                    ['pairsam-parse-sort',['0'],150],
                    ['pairsam-merge',['0'],150],
                    ['pairsam-markasdup',['0'],150],
                    ['pairsam-filter',['0'],150],
                    ['addfragtopairs',['0'],150],
                    ['pairs-patch',['0'],150],
                    ['hi-c-processing-partb set',['0'],150],
                    ['hi-c-processing-partb exp',['0'],150],
                    ['hi-c-processing-partc set',['0'],150],
                    ['hi-c-processing-partc exp',['0'],150]
                   ]
workflow_names = [i[0] for i in workflow_details]

deleted_wfr_no = 0
files_with_deleted_wfr = 0
files = [i['uuid'] for i in ff_utils.get_metadata('files' , connection=ff)['@graph']]
print len(files), 'files in the system'
deleted_wfrs = []
counter = 0
del_md5 = 0
del_qc = 0
deleted_output = 0
for a_file in files:
    counter += 1
    if counter % 100 == 0:
        print counter, files_with_deleted_wfr
    raw_file = ff_utils.get_metadata(a_file, connection = ff, frame='embedded')
    deleted_wf = False
    wfr_report = []
    wfrs = raw_file.get('workflow_run_inputs')
    
    # Delete wfrs if file is deleted
    if raw_file['status'] == 'deleted':
        if delete_workflows.lower() in ['y', 'yes']:
            # clean deleted files of md5 and qc metrics
            for a_field in ['content_md5sum', 'md5sum']:  
                if raw_file.get(a_field):
                    ff_utils.delete_field(raw_file, a_field, connection=ff)
                    del_md5 += 1
            if raw_file.get('quality_metric'):
                qc_uuid = raw_file['quality_metric']['uuid']
                ff_utils.delete_field(raw_file, 'quality_metric', connection=ff)
                # delete quality metrics object
                patch_data = {'status': "deleted"}
                ff_utils.patch_metadata(patch_data, obj_id=qc_uuid ,connection=ff)
                del_qc += 1
        # delete all workflows for deleted files
        if not wfrs:
            continue
        else:
            wfr_report = get_wfr_report(wfrs)
            for wfr_to_del in wfr_report:
                if wfr_to_del['status'] != 'deleted':
                    if wfr_to_del['wfr_name'] not in workflow_names:
                        print('Unlisted Workflow', wfr_to_del['wfr_name'], 'deleted file workflow', wfr_to_del['wfr_uuid'], raw_file['accession'])
                    deleted_wf = True
                    deleted_wfr_no += 1
                    
                    ####################################################
                    ## TEMPORARY PIECE##################################
                    if wfr_to_del['status'] == 'released to project':
                        print('saved from deletion', wfr_to_del['wfr_name'], 'deleted file workflow', wfr_to_del['wfr_uuid'], raw_file['accession'])
                        continue
                    if wfr_to_del['status'] == 'released':
                        print('delete released!!!!!', wfr_to_del['wfr_name'], 'deleted file workflow', wfr_to_del['wfr_uuid'], raw_file['accession'])
                        continue  
                    #####################################################
                    
                    print(wfr_to_del['wfr_name'], 'deleted file workflow', wfr_to_del['wfr_uuid'], raw_file['accession'])
                    if delete_workflows.lower() in ['y', 'yes']:
                        patch_data = {'description': "This workflow run is deleted", 'status': "deleted"}
                        deleted_wfrs.append(wfr_to_del['wfr_uuid'])
                        ff_utils.patch_metadata(patch_data, obj_id=wfr_to_del['wfr_uuid'] ,connection=ff)
                        # delete output files of the deleted workflow run
                        if wfr_to_del['outputs']:
                            for out_file in wfr_to_del['outputs']:
                                deleted_output += 1
                                ff_utils.patch_metadata({'status': "deleted"}, obj_id=out_file ,connection=ff)
       
                
    else:
        # get a report on all workflow_runs
        if not wfrs:
            continue
        else:
            wfr_report = get_wfr_report(wfrs)
            # printTable(wfr_report, ['wfr_name', 'run_time', 'wfr_rev', 'run_time', 'wfr_status'])
            
            # check if any unlisted wfr in report
            my_wfr_names = [i['wfr_name'] for i in wfr_report]
            unlisted = [x for x in my_wfr_names if x not in workflow_names]
            if unlisted:
                print('Unlisted Workflow', unlisted, 'skipped in', raw_file['accession'])
                    
            for wf_name,accepted_rev,accepted_run_time in workflow_details:
                #for each type of worklow make a list of old ones, and patch status and description
                sub_wfrs = [i for i in wfr_report if i['wfr_name'] == wf_name]
                if sub_wfrs:
                    active_wfr = sub_wfrs[-1]
                    old_wfrs = sub_wfrs [:-1]
                    # check the status of the most recent workflow
                    if active_wfr['wfr_status'] != 'complete':
                        if active_wfr['wfr_status'] in ['running', 'started'] and active_wfr['run_time'] < accepted_run_time:
                            print wf_name,'still running for', a_file
                        else:
                            old_wfrs.append(active_wfr)
                    elif active_wfr['wfr_rev'] not in accepted_rev:
                        old_wfrs.append(active_wfr)
                    if old_wfrs:
                        for wfr_to_del in old_wfrs:
                            if wfr_to_del['status'] != 'deleted':
                                deleted_wf = True
                                deleted_wfr_no += 1 
                                
                                ####################################################
                                ## TEMPORARY PIECE
                                if wfr_to_del['status'] == 'released to project':
                                    print('saved from deletion',wfr_to_del['wfr_name'], 'old style or dub', wfr_to_del['wfr_uuid'], raw_file['accession'])
                                    continue
                                if wfr_to_del['status'] == 'released':
                                    print('delete released????',wfr_to_del['wfr_name'], 'old style or dub', wfr_to_del['wfr_uuid'], raw_file['accession'])
                                    continue
                                ####################################################

                                print(wfr_to_del['wfr_name'], 'old style or dub', wfr_to_del['wfr_uuid'], raw_file['accession'])
                                
                                if delete_workflows.lower() in ['y', 'yes']:
                                    patch_data = {'description': "This workflow run is deleted", 'status': "deleted"}
                                    deleted_wfrs.append(wfr_to_del['wfr_uuid'])
                                    
                                    ff_utils.patch_metadata(patch_data, obj_id=wfr_to_del['wfr_uuid'] ,connection=ff)
                                    # delete output files of the deleted workflow run
                                    if wfr_to_del['outputs']:
                                        for out_file in wfr_to_del['outputs']:
                                            deleted_output += 1
                                            ff_utils.patch_metadata({'status': "deleted"}, obj_id=out_file ,connection=ff)
    if deleted_wf:
        files_with_deleted_wfr += 1

if delete_workflows.lower() in ['y', 'yes']:
    print str(deleted_wfr_no),"workflowruns from", str(files_with_deleted_wfr), "files deleted"
else:
    print str(deleted_wfr_no),"workflowruns from", str(files_with_deleted_wfr), "files need to be deleted"

print len(deleted_wfrs)
print del_md5, 'md5 fields deleted'
print del_qc, 'qc metrics deleted'
print deleted_output, 'deleted output files'
print 'finished at', datetime.utcnow()

started at 2017-10-18 17:36:11.348990
Do you want to delete old workflowruns (if not, only report will be displayed (y/n))
5376 files in the system
(u'addfragtopairs', 'old style or dub', u'b6392a8b-dc26-4c47-9f33-cc14ecdca3f7', u'4DNFIUJZ4X9G')
(u'addfragtopairs', 'old style or dub', u'0dd6eabc-120f-48da-b512-28386c6fb75f', u'4DNFIUJZ4X9G')
(u'addfragtopairs', 'old style or dub', u'018707cb-b9fb-4459-8ff9-6aede15ec235', u'4DNFIUJZ4X9G')
(u'addfragtopairs', 'old style or dub', u'582a7b2a-2d08-475e-a1f5-a215e4747937', u'4DNFIUJZ4X9G')
(u'addfragtopairs', 'old style or dub', u'86187886-3306-4a46-8460-e2bb2a00645a', u'4DNFIUJZ4X9G')
(u'addfragtopairs', 'old style or dub', u'b9df751b-9c5f-4985-b9ce-191ae2a08b03', u'4DNFIUJZ4X9G')
(u'addfragtopairs', 'old style or dub', u'36b5b4ea-34a1-43fe-a84e-65fb170e24e0', u'4DNFIUJZ4X9G')
(u'addfragtopairs', 'old style or dub', u'2e43395c-dd21-4699-b561-1e42cce3683e', u'4DNFIUJZ4X9G')
(u'addfragtopairs', 'old style or dub', u'c750211c-15f9-4d8b-a76f-2d

KeyboardInterrupt: 